Number of samples : 17570

Number of features : 5

Target : total_sales(mil) = total sales in million

Description : dataset of video game with metadata (console, genre, publisher, developer, release date), a critical score and sales(total and regional).

Problème : predict total_sales(mil) in fonction of features (console/genre/publisher/developer/release_year),
without using sales regional to avoid leak of information.
(we remove critical score because there is missing 90% of data and this is unusable)

Intérêt industrie : estimation of comercial potential / understanding factors associated with sales

To start we load the csv with all data information for the video games to read it and get the data

In [ ]:
import pandas as pd

df = pd.read_csv("regression_datasets/VideoGames_Sales.csv", sep=";")
print(df.shape)
print(df.columns)

(64016, 12)
Index(['title', 'console', 'genre', 'publisher', 'developer', 'critic_score',
       ' total_sales(mil) ', ' na_sales(mil) ', ' jp_sales(mil) ',
       ' pal_sales(mil) ', ' other_sales(mil) ', 'release_date'],
      dtype='object')


after we need to remove unecessary column like the title for avoid learn title by heart instead of tentency, sales to avoid leak of information.
Then we applies a log function to applied symetry of sales

In [56]:
def convert_sales_in_number(df):
    df = df.copy()
    df.columns = df.columns.str.strip()

    sales_columns = [
        "na_sales(mil)",
        "jp_sales(mil)",
        "pal_sales(mil)",
        "other_sales(mil)",
        "total_sales(mil)",
    ]

    for c in sales_columns:
        if c in df.columns:
            s = df[c].astype(str).str.strip()
            s = s.str.replace("\u00A0", "", regex=False)
            s = s.str.replace(r"[^0-9\-\.,]", "", regex=True)
            s = s.str.replace(",", ".", regex=False)

            df[c] = pd.to_numeric(s, errors="coerce")

    return df

We create a function to convert data of sales into number because there are in string

In [57]:
import pandas as pd
import numpy as np

df.columns = df.columns.str.strip()

df = convert_sales_in_number(df)

target = "total_sales(mil)"

print("NaN target after conversion:", df[target].isna().sum(), "out of", len(df))

df = df.dropna(subset=[target])
df = df[df[target] >= 0]

feature_cols = ["console", "genre", "publisher", "developer", "release_date"]
X = df[feature_cols].copy()

X["release_date"] = pd.to_datetime(X["release_date"], errors="coerce", dayfirst=True)
X["release_year"] = X["release_date"].dt.year
X = X.drop(columns=["release_date"])

y = np.log1p(df[target])

print("Samples:", len(df))
print("X shape:", X.shape)
print("y shape:", y.shape)
print("y NaN:", np.isnan(y).sum())

print("Samples:", len(df))
print("Raw features:", X.shape[1])
print("Target:", target, "(log1p transform applied)")
print("Missing values per column:\n", X.isna().sum())


NaN target after conversion: 46446 out of 64016
Samples: 17570
X shape: (17570, 5)
y shape: (17570,)
y NaN: 0
Samples: 17570
Raw features: 5
Target: total_sales(mil) (log1p transform applied)
Missing values per column:
 console          0
genre            0
publisher        0
developer        2
release_year    62
dtype: int64


The csv file contains 64016 entries but the target variable total_sales(mil) is missing for 46446 rows. After cleaning (converting sales to numeric and removing missing targets), 17570 usable sample remain for training

We implement ridge regression with descent of gradiant

In [ ]:
class RidgeGD:
    def __init__(self, lr=1e-3, epochs=3000, l2=1e-3, clip=1.0, verbose=200):
        self.lr = lr
        self.epochs = epochs
        self.l2 = l2
        self.clip = clip
        self.verbose = verbose
        self.w = None
        self.b = 0.0

    def fit(self, X, y):
        n, d = X.shape
        self.w = np.zeros(d, dtype=np.float64)
        self.b = 0.0

        for t in range(1, self.epochs + 1):
            yhat = X @ self.w + self.b
            err = yhat - y

            grad_w = (2.0/n) * (X.T @ err) + 2.0 * self.l2 * self.w
            grad_b = (2.0/n) * np.sum(err)

            if self.clip is not None:
                grad_w = np.clip(grad_w, -self.clip, self.clip)
                grad_b = float(np.clip(grad_b, -self.clip, self.clip))

            self.w -= self.lr * grad_w
            self.b -= self.lr * grad_b

            if self.verbose and t % self.verbose == 0:
                loss = np.mean(err**2) + self.l2 * np.sum(self.w**2)
                if not np.isfinite(loss):
                    print("Loss became non-finite (diverged). Try smaller lr.")
                    break
                print(f"epoch {t} | loss {loss:.6f}")

        return self

    def predict(self, X):
        return X @ self.w + self.b

Now we prepare data, separate train/test and evaluate the regression modele.
First we encode one-hot the categorical variables and implement missing values.
then we convert in array and separate train/test randomly
after define metrics we use a baseline "Dummy" to predict mean
and finaly we train and predict our model.
we need to return to the original sales scale (reverse log1p) and we analyse errors by sales ranges.

In [ ]:
X_oh = pd.get_dummies(X, columns=["console","genre","publisher","developer"], dummy_na=False)

X_oh["release_year"] = X_oh["release_year"].fillna(X_oh["release_year"].median())

mu = X_oh["release_year"].mean()
sigma = X_oh["release_year"].std()
X_oh["release_year"] = (X_oh["release_year"] - mu) / (sigma + 1e-8)

X_mat = X_oh.to_numpy(dtype=np.float64)
y_vec = np.asarray(y, dtype=np.float64)

rng = np.random.default_rng(42)
idx = np.arange(len(y_vec))
rng.shuffle(idx)

split = int(0.8 * len(idx))
train_idx, test_idx = idx[:split], idx[split:]

X_train, X_test = X_mat[train_idx], X_mat[test_idx]
y_train, y_test = y_vec[train_idx], y_vec[test_idx]

def mae(y, yhat):
    return np.mean(np.abs(y - yhat))

def rmse(y, yhat):
    return np.sqrt(np.mean((y - yhat) ** 2))

def r2(y, yhat):
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    return 1 - ss_res/ss_tot

y_mean = np.mean(y_train)
dummy_pred = np.full_like(y_test, y_mean)

print("DUMMY | MAE(log):", mae(y_test, dummy_pred),
      "| RMSE(log):", rmse(y_test, dummy_pred),
      "| R2:", r2(y_test, dummy_pred))

model = RidgeGD(lr=1e-3, epochs=3000, l2=1e-3, clip=1.0, verbose=200).fit(X_train, y_train)
pred = model.predict(X_test)

print("RIDGE_GD | MAE(log):", mae(y_test, pred),
      "| RMSE(log):", rmse(y_test, pred),
      "| R2:", r2(y_test, pred))

y_true_m = np.expm1(y_test)
y_pred_m = np.expm1(pred)

print("Original | MAE:", mae(y_true_m, y_pred_m), "million",
      "| RMSE:", rmse(y_true_m, y_pred_m), "million")

abs_err = np.abs(y_pred_m - y_true_m)

bins = [0, 0.1, 0.5, 1, 5, 20]
labels = ["<0.1", "0.1-0.5", "0.5-1", "1-5", "5-20"]

groups = pd.cut(y_true_m, bins=bins, labels=labels, include_lowest=True)

for lab in labels:
    mask = (groups == lab)
    if mask.sum() > 0:
        print(lab, "count=", int(mask.sum()), "MAE(mil)=", float(abs_err[mask].mean()))


DUMMY | MAE(log): 0.21969028843267657 | RMSE(log): 0.32092038760906716 | R2: -1.197554618825869e-05
epoch 200 | loss 0.123503
epoch 400 | loss 0.107607
epoch 600 | loss 0.100989
epoch 800 | loss 0.097982
epoch 1000 | loss 0.096397
epoch 1200 | loss 0.095386
epoch 1400 | loss 0.094621
epoch 1600 | loss 0.093971
epoch 1800 | loss 0.093386
epoch 2000 | loss 0.092845
epoch 2200 | loss 0.092337
epoch 2400 | loss 0.091858
epoch 2600 | loss 0.091405
epoch 2800 | loss 0.090975
epoch 3000 | loss 0.090566
RIDGE_GD | MAE(log): 0.2020958216879382 | RMSE(log): 0.30247484564477883 | R2: 0.11163976096675698
Original | MAE: 0.3303186862558893 million | RMSE: 0.7981263602284683 million
<0.1 count= 1421 MAE(mil)= 0.20541164215922808
0.1-0.5 count= 1411 MAE(mil)= 0.11274943212693145
0.5-1 count= 390 MAE(mil)= 0.38118609446358626
1-5 count= 270 MAE(mil)= 1.4969100044779777
5-20 count= 22 MAE(mil)= 7.133281509726325


We can see that we have the loss who decrease reguliary showing convergence. The model exceeds the baseline (R²≈0.11 vs ≈0), but remains limited by its linear nature and the high variability of sales.

here We research the hyperparametre (tuning) for the model and  train the final model with the best found hyperparametre.
Creation of train and score function with hyperparametre and 600 epoch and return MAE score.
Then we make a loop with hyperparametre grid and see the score.
After found the best result we make the final training with best hyperparametre

In [60]:
def train_and_score(lr, l2, epochs=600):
    m = RidgeGD(lr=lr, epochs=epochs, l2=l2, clip=1.0, verbose=0).fit(X_train, y_train)
    p = m.predict(X_test)
    return mae(y_test, p)

best = None
for l2 in [0.0, 1e-4, 1e-3, 1e-2]:
    for lr in [1e-4, 5e-4, 1e-3]:
        score = train_and_score(lr, l2, epochs=600)
        print("lr", lr, "l2", l2, "MAE(log)", score)
        if best is None or score < best[0]:
            best = (score, lr, l2)

print("Best quick:", best)

_, best_lr, best_l2 = best
model = RidgeGD(lr=best_lr, epochs=3000, l2=best_l2, clip=1.0, verbose=200).fit(X_train, y_train)
pred = model.predict(X_test)
print("Final MAE(log):", mae(y_test, pred))


lr 0.0001 l2 0.0 MAE(log) 0.22039378571476614
lr 0.0005 l2 0.0 MAE(log) 0.1911682945010726
lr 0.001 l2 0.0 MAE(log) 0.19474913648057812
lr 0.0001 l2 0.0001 MAE(log) 0.22039380949023016
lr 0.0005 l2 0.0001 MAE(log) 0.19116844407402192
lr 0.001 l2 0.0001 MAE(log) 0.19474920313028418
lr 0.0001 l2 0.001 MAE(log) 0.22039402346076287
lr 0.0005 l2 0.001 MAE(log) 0.1911697899645993
lr 0.001 l2 0.001 MAE(log) 0.19474980299251266
lr 0.0001 l2 0.01 MAE(log) 0.22039616231064285
lr 0.0005 l2 0.01 MAE(log) 0.1911832225787399
lr 0.001 l2 0.01 MAE(log) 0.19475583539129487
Best quick: (np.float64(0.1911682945010726), 0.0005, 0.0)
epoch 200 | loss 0.138790
epoch 400 | loss 0.123454
epoch 600 | loss 0.113763
epoch 800 | loss 0.107585
epoch 1000 | loss 0.103597
epoch 1200 | loss 0.100973
epoch 1400 | loss 0.099202
epoch 1600 | loss 0.097965
epoch 1800 | loss 0.097064
epoch 2000 | loss 0.096377
epoch 2200 | loss 0.095825
epoch 2400 | loss 0.095362
epoch 2600 | loss 0.094958
epoch 2800 | loss 0.094593
epoch

We can see that l2 doesn't improve MAE, the best result is l2 = 0. that mean that for this split and these feature, the regularization doesn't provide a net gain.
so a pre-turning (600 epoch) on a grid of hyperparametre indicates that lr=5e-4 and l2=0 minimize MAE and are the best hyperparametre. the Long training (3000 epoch) converge with decreasing loss and obtain MAE≈0.205.
So we see that the linear model tends toward an average estimate

Here we return to original scales and calculate the absolute error by sample. 
After we select 10 example randomly for the test and print the prediction for these. 
Then we select 10 worst error and print these.

In [61]:
y_true_m = np.expm1(y_test)
y_pred_m = np.expm1(pred)
abs_err = np.abs(y_pred_m - y_true_m)

rng = np.random.default_rng(0)
choices = rng.choice(len(y_test), size=10, replace=False)

print("\n--- Random examples (general behavior) ---")
for i in choices:
    print(f"True: {y_true_m[i]:.2f} mil | Pred: {y_pred_m[i]:.2f} mil | AbsErr: {abs_err[i]:.2f} mil")
    
print("\n--- 10 worst predictions errors (limitations / outliers) ---")
worst = np.argsort(-abs_err)[:10]
for i in worst:
    print(f"True: {y_true_m[i]:.2f} mil | Pred: {y_pred_m[i]:.2f} mil | AbsErr: {abs_err[i]:.2f} mil")



--- Random examples (general behavior) ---
True: 0.31 mil | Pred: 0.40 mil | AbsErr: 0.09 mil
True: 8.88 mil | Pred: 0.27 mil | AbsErr: 8.61 mil
True: 0.01 mil | Pred: 0.25 mil | AbsErr: 0.24 mil
True: 1.22 mil | Pred: 0.33 mil | AbsErr: 0.89 mil
True: 0.29 mil | Pred: 0.21 mil | AbsErr: 0.08 mil
True: 0.02 mil | Pred: 0.21 mil | AbsErr: 0.19 mil
True: 0.14 mil | Pred: 0.20 mil | AbsErr: 0.06 mil
True: 0.04 mil | Pred: 0.30 mil | AbsErr: 0.26 mil
True: 1.48 mil | Pred: 0.26 mil | AbsErr: 1.22 mil
True: 0.02 mil | Pred: 0.29 mil | AbsErr: 0.27 mil

--- 10 worst predictions errors (limitations / outliers) ---
True: 14.82 mil | Pred: 0.35 mil | AbsErr: 14.47 mil
True: 13.53 mil | Pred: 0.36 mil | AbsErr: 13.17 mil
True: 10.41 mil | Pred: 0.34 mil | AbsErr: 10.07 mil
True: 9.96 mil | Pred: 0.30 mil | AbsErr: 9.66 mil
True: 9.32 mil | Pred: 0.28 mil | AbsErr: 9.04 mil
True: 8.88 mil | Pred: 0.27 mil | AbsErr: 8.61 mil
True: 8.01 mil | Pred: 0.36 mil | AbsErr: 7.65 mil
True: 7.26 mil | Pred

We can see the biggest mistake involve very high sales and works better on small/medium sales.
The model captures an average estimate but strugglies to predict very big success. The example show a systematic underestimation of high-selling game, because these game depend on unobserved factors (licensing, marketing, and brand awarness) and the linear model tends towards an average estimate.